In [1]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd

# Load the dataset
file_path = "C:/Users/S41919/Downloads/Data Engineering/Data Engineering/data - sample.xlsx"  # Update this if needed
attendance_df = pd.read_excel(file_path)

# Convert date column to datetime
attendance_df['attendance_date'] = pd.to_datetime(attendance_df['attendance_date'])

# Sort data by student_id and date
attendance_df = attendance_df.sort_values(by=['student_id', 'attendance_date'])

def get_latest_absence_streaks(attendance_df):
    latest_streaks = []
    
    for student_id, group in attendance_df.groupby('student_id'):
        group = group.reset_index(drop=True)
        group['consecutive_group'] = (group['attendance_date'].diff().dt.days != 1).cumsum()
        
        # Extract only absent records
        absences = group[group['status'] == 'Absent']
        
        # Group consecutive absences
        streaks = absences.groupby('consecutive_group').agg(
            absence_start_date=('attendance_date', 'first'),
            absence_end_date=('attendance_date', 'last'),
            total_absent_days=('attendance_date', 'count')
        ).reset_index(drop=True)
        
        streaks['student_id'] = student_id
        
        # Filter only streaks with more than 3 absences
        valid_streaks = streaks[streaks['total_absent_days'] > 3]
        
        if not valid_streaks.empty:
            latest_streaks.append(valid_streaks.iloc[-1])  # Take latest streak
    
    return pd.DataFrame(latest_streaks)

# Get final corrected output
final_absence_streaks = get_latest_absence_streaks(attendance_df)[['student_id', 'absence_start_date', 'absence_end_date', 'total_absent_days']]

# Display final corrected output
final_absence_streaks


,student_id,absence_start_date,absence_end_date,total_absent_days
1,101,2024-03-09,2024-03-14,4
3,107,2024-03-20,2024-03-25,5
4,108,2024-11-02,2024-11-05,4


In [9]:
import pandas as pd
from tabulate import tabulate

# Sample data for student absence records
data = {
    'student_id': [101, 102, 103],
    'absence_start_date': ['2024-03-01', '2024-03-02', '2024-03-05'],
    'absence_end_date': ['2024-03-04', '2024-03-05', '2024-03-09'],
    'total_absent_days': [4, 4, 5],
    'student_name': ['Alice Johnson', 'Bob Smith', 'Charlie Brown'],
    'email': ['alice_parent@example.com', 'bob_parent@example.com', None]
}

# Convert the dictionary into a Pandas DataFrame
df = pd.DataFrame(data)

# Convert dates to "DD-MM-YYYY" format
df['absence_start_date'] = pd.to_datetime(df['absence_start_date']).dt.strftime('%d-%m-%Y')
df['absence_end_date'] = pd.to_datetime(df['absence_end_date']).dt.strftime('%d-%m-%Y')

# Generate message column
df['msg'] = df.apply(lambda row: (
    f"Dear Parent, your child {row['student_name']} was absent from {row['absence_start_date']} to {row['absence_end_date']} for {row['total_absent_days']} days. "
    f"Please ensure their attendance improves."
) if row['email'] else "None", axis=1)

# Select required columns for final display
df = df[['student_id', 'absence_start_date', 'absence_end_date', 'total_absent_days', 'email', 'msg']]

# Display the table in a well-formatted manner
print(tabulate(df, headers='keys', tablefmt='grid'))


+----+--------------+----------------------+--------------------+---------------------+--------------------------+-------------------------------------------------------------------------------------------------------------------------------------+
|    |   student_id | absence_start_date   | absence_end_date   |   total_absent_days | email                    | msg                                                                                                                                 |
+====+==============+======================+====================+=====================+==========================+=====================================================================================================================================+
|  0 |          101 | 01-03-2024           | 04-03-2024         |                   4 | alice_parent@example.com | Dear Parent, your child Alice Johnson was absent from 01-03-2024 to 04-03-2024 for 4 days. Please ensure their attendance improves. |
+---